In [2]:
import numpy as np
import pandas as pd
import gisttools as gt
import matplotlib.pyplot as plt

In [3]:
import os
os.getpid()

2100308

In [ ]:
gist = lo ad_gist_ file (
' gist . dat ' , struct = ' solute - centered . pdb ' ,

    'gist.dat', struct='solute-centered.pdb',


In [6]:
!echo $PWD

/home/vah/GIST_Tutorial/gist-tutorial/code


In [5]:
!cd gist-tutorial/code/

/bin/bash: line 1: cd: gist-tutorial/code/: No such file or directory


In [ ]:
compl = gt.gist.load_gist_file('complex/gist/gist.dat', struct='complex/gist/complex-centered.pdb')
print(compl.n_frames, compl.rho0)

In [ ]:
biotin = gt.gist.load_gist_file('biotin/gist/gist.dat', struct='biotin/gist/biotin-centered.pdb')
print(biotin.n_frames, biotin.rho0)

In [ ]:
strept = gt.gist.load_gist_file('streptavidin/gist/gist.dat', struct='streptavidin/gist/streptavidin-centered.pdb')
print(strept.n_frames, strept.rho0)

In [ ]:
biotin.eww_ref = biotin.detect_reference_value()
print("Biotin:", biotin.eww_ref)
strept.eww_ref = strept.detect_reference_value()
print("Streptavidin:", strept.eww_ref)
compl.eww_ref = compl.detect_reference_value()
print("Complex:", compl.eww_ref)

In [ ]:
biotin.eww_ref = biotin.detect_reference_value()

In [ ]:
def reference_entropy(gf):
    if 'dTSsix_unref_norm' not in gf.data.columns:
        gf['dTSsix_unref_norm'] = gf['dTSsix_norm']
        gf['dTSsix_unref_dens'] = gf['dTSsix_dens']
    refval = gf.detect_reference_value('dTSsix_unref_dens')
    gf['dTSsix_norm'] = gf.get_referenced('dTSsix_unref_norm', refval)
    gf['dTSsix_dens'] = gf.get_referenced('dTSsix_unref_dens', refval)

reference_entropy(biotin)
reference_entropy(strept)
reference_entropy(compl)

In [ ]:
biotin_mask = 'resname BTN and not element H'
strept_mask = 'not resname BTN and not resname WAT and not element H'

In [ ]:
import mdtraj as md

In [ ]:
biotin['A06_dens'] = biotin['Eall_dens'] - 0.6 * biotin['dTSsix_dens']
strept['A06_dens'] = strept['Eall_dens'] - 0.6 * strept['dTSsix_dens']
compl['A06_dens'] = compl['Eall_dens'] - 0.6 * compl['dTSsix_dens']

In [ ]:
col = 'A06_dens'
def select(traj, sel):
    "Slice a Trajectory by selection mask."
    return traj.atom_slice(traj.top.select(sel))
# we multiply by 10 to convert nm to Angstrom.
compl_x = select(compl.struct, biotin_mask).xyz[0] * 10.
biotin_x = select(biotin.struct, biotin_mask).xyz[0] * 10.
aligned = compl.struct[:].superpose(strept.struct, \
    atom_indices=strept.struct.top.select(strept_mask))
aligned = select(aligned, biotin_mask)
strept_x = aligned.xyz[0] * 10.

bins, biotin_rdf = biotin.rdf( \
    col, centers=biotin_x, bins=100, rmax=24)
bins, strept_rdf = strept.rdf( \
    col, centers=strept_x, bins=100, rmax=24)
bins, compl_rdf = compl.rdf( \
    col, centers=compl_x, bins=100, rmax=24)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
print(col)

cutoff = 12
ax1.plot(bins, np.cumsum(biotin_rdf), label='biotin')
ax1.plot(bins, np.cumsum(strept_rdf), label='streptavidin')
ax1.plot(bins, np.cumsum(compl_rdf), label='complex')
ax1.legend()
ax1.axvline(cutoff, color='k', linestyle='--')
ax1.set_xlabel('distance to biotin [Å]')
ax1.set_ylabel('$\Delta G$ contributions [Å]')
ax1.grid()

difference = compl_rdf - biotin_rdf - strept_rdf
cutoff = 12
integral = difference[bins < cutoff].sum() + avg_interaction
print("Integral = {}".format(integral))
ax2.plot(bins, np.cumsum(difference))
ax2.axvline(cutoff, color='k', linestyle='--')
ax2.set_xlabel('distance to biotin [Å]')
ax2.set_ylabel('$\Delta \Delta G$ [kcal/mol]')
ax2.grid()

plt.tight_layout()

plt.savefig('../manuscript/figures/deltaG-difference.png', dpi=300)
plt.show()

In [ ]:
avg_interaction = -98.666170

In [ ]:
def integrate(rdf):
    return rdf[bins < cutoff].sum()# + avg_interaction

In [ ]:
result = pd.Series({
    'complex': integrate(compl_rdf),
    'strept': integrate(strept_rdf),
    'biotin': integrate(biotin_rdf),
    'diff': integrate(difference)
})
result

In [ ]:
# only relevant for dTSsix
result * 0.6

In [ ]:
print('Integrals + interaction energy')
print(f"Complex: {integrate(compl_rdf)+avg_interaction}, Strept: {integrate(strept_rdf)+avg_interaction}, Biotin: {integrate(biotin_rdf)+avg_interaction}")

In [ ]:
energy = pd.read_csv('complex/gist/interaction_energy/energy.dat', delim_whitespace=True, names='frame complex strept biotin diff'.split(), skiprows=1).drop(columns='frame')

In [ ]:
energy.mean()

In [ ]:
total = energy.mean() + result
total

In [ ]:
total['complex'] - total['strept'] - total['biotin'] - total['diff']